# Import libraries

In [78]:
import numpy as np # calculations with arrays
import pandas as pd # user-friendly DataFrames for data representation
import sklearn # machine learning algorithms
from sklearn import ensemble, linear_model
from sklearn.metrics import log_loss
from sklearn import cross_validation
from sklearn import grid_search
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import DictVectorizer as DV
import xgboost as xgboost
import matplotlib.pyplot as plt # import plot functions
# necessary to plot in jupyter notebook:
%matplotlib inline
import seaborn as sns # make plots beautiful
from sklearn.grid_search import GridSearchCV

# Load data

In [79]:
train = pd.read_csv('train2.csv')
test = pd.read_csv('test2.csv')
sample_submission = pd.read_csv('sample_submission.csv')

# Data

In [80]:
train.head(3)

,year,day,team1,team2,score1,score2,target
0,2998,19,317,131,336,278,True
1,2998,28,61,29,301,259,True
2,2998,28,110,141,359,267,True


In [81]:
train.shape

(101609, 7)

In [82]:
test.head(3)

,Id,year,team1,team2
0,0,3021,363,161
1,1,3021,286,2
2,2,3020,232,52


In [83]:
sample_submission.head(3)

,Id,target
0,0,0.5
1,1,0.5
2,2,0.5


In [84]:
print 'train.shape = ', train.shape
print 'test.shape = ', test.shape
print 'sample_submission.shape = ', sample_submission.shape

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-84-4fd11fc4112d>, line 1)

# Work with categorical features

# Make dataset for cross-vallidation using One-Hot-Encoding

In [85]:
enc = OneHotEncoder(sparse=False)

In [86]:
train_team1 = pd.DataFrame(enc.fit_transform(train[['team1']]))

In [87]:
print 'train_team1.shape = ', train_team1.shape

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-87-420bd4dd2b48>, line 1)

In [88]:
train_team2 = pd.DataFrame(enc.transform(train[['team2']]))

In [12]:
print 'train_team2.shape = ', train_team2.shape

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-12-220f88f414c6>, line 1)

In [89]:
train_teams = pd.concat([train_team1, train_team2], axis=1)

In [14]:
print 'train_team1 + train_team2 = ', train_teams.shape

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-14-86ee706d0bfd>, line 1)

In [15]:
test_team1 = pd.DataFrame(enc.transform(test[['team1']]))

In [16]:
print 'test_team1 = ', test_team1.shape

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-16-63a9959574ec>, line 1)

In [17]:
test_team2 = pd.DataFrame(enc.transform(test[['team2']]))

In [18]:
print 'test_team2 = ', test_team2.shape

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-18-0f3922acb5e1>, line 1)

In [19]:
test_teams = pd.concat([test_team1, test_team2], axis=1)

In [20]:
print 'test_teams.shape = ', test_teams.shape

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-20-23466d088f6b>, line 1)

# Feature engenering

### Добавили среднее число очков: для команды team1 – среднее количество очков, набранных дома; для команды team2 – среднее количество очков, набранных в гостях

In [21]:
df_train = train
df_test = test

In [22]:
means_score = {}
for index_team in list(df_train.team1.unique()):
    df_for_team1 = df_train[df_train['team1'] == index_team]
    df_for_team2 = df_train[df_train['team2'] == index_team]
    means_score[index_team] = [int(df_for_team1.score1.mean()), int(df_for_team2.score2.mean())]
#print(means_score)
#for c in means_score:
#    print(means_score[c])

In [23]:
teams_not_train = []
for index_team in list(df_test.team1.unique()):
    if index_team not in list(df_train.team1.unique()):
        teams_not_train.append(index_team)
for index_team in list(df_test.team2.unique()):
    if index_team not in list(df_train.team1.unique()):
        teams_not_train.append(index_team)

In [24]:
for index_team in teams_not_train:
    means_score[index_team] = [df_train.score1.mean(), df_train.score2.mean()]

In [25]:
features_to_concat_train = []
for year, day, team1, team2, _, __, ___ in df_train.values:
    features_to_concat_train.append([means_score[team1][0], means_score[team2][1]])
features_to_concat_test = []
for Id, year, team1, team2 in df_test.values:
    features_to_concat_test.append([means_score[team1][0], means_score[team2][1]])

In [26]:
df_feature_to_concat_train = pd.DataFrame(data=features_to_concat_train, columns=['mean_home_score', 'mean_away_score'])

In [27]:
df_feature_to_concat_test = pd.DataFrame(data=features_to_concat_test, columns=['mean_home_score', 'mean_away_score'])

### Добавляем процент побед: для team1 – процент побед дома; для team2 – процент побед в гостях

In [28]:
percentage_wins = {}
for index_team in list(df_train.team1.unique()):
    df_for_team_home = df_train[df_train['team1'] == index_team]
    df_for_team_away = df_train[df_train['team2'] == index_team]
    percentage_wins[index_team] = [df_for_team_home.target.mean(), 1 - df_for_team_away.target.mean()]

In [29]:
for index_team in teams_not_train:
    percentage_wins[index_team] = [df_train.target.mean(), 1 - df_train.target.mean()]

In [30]:
percentage_of_wins_train = []
for year, day, team1, team2, _, __, ___ in df_train.values:
    percentage_of_wins_train.append([percentage_wins[team1][0], percentage_wins[team2][1]])
percentage_of_wins_test = []
for Id, year, team1, team2 in df_test.values:
    percentage_of_wins_test.append([percentage_wins[team1][0], percentage_wins[team2][1]])

In [31]:
df_percentage_of_wins_train = pd.DataFrame(data=percentage_of_wins_train, columns=['percentage_wins_home', 'percentage_wins_away'])

In [32]:
df_percentage_of_wins_test = pd.DataFrame(data=percentage_of_wins_test, columns=['percentage_wins_home', 'percentage_wins_away'])

   # Make X_train, X_test, y_train, y_test

### Делаем X_train

In [33]:
X_train_all_feat = pd.concat([train, df_feature_to_concat_train, df_percentage_of_wins_train], axis=1)

In [34]:
print 'X_train_all_feat.shape = ', X_train_all_feat.shape

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-34-adf97a8cc3c5>, line 1)

In [35]:
X_train_all_feat.head(2)

,year,day,team1,team2,score1,score2,target,mean_home_score,mean_away_score,percentage_wins_home,percentage_wins_away
0,2998,19,317,131,336,278,True,289,285,0.688073,0.640118
1,2998,28,61,29,301,259,True,273,276,0.503333,0.556962


In [36]:
y_train =  X_train_all_feat[['target']]

In [37]:
X_train_not_scaled = X_train_all_feat.drop(['year', 'mean_home_score', 'mean_away_score', 'day', 'team1', 'team2','score1', 'score2', 'target'], axis=1)

In [38]:
X_train_not_scaled.head(2)

,percentage_wins_home,percentage_wins_away
0,0.688073,0.640118
1,0.503333,0.556962


In [39]:
print 'X_train_not_scaled.shape = ', X_train_not_scaled.shape

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-39-316710af31d9>, line 1)

In [40]:
print 'y_train.shape = ', y_train.shape

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-40-fdab9f5dd08e>, line 1)

In [41]:
X_test_all_feat = pd.concat([test, df_feature_to_concat_test, df_percentage_of_wins_test], axis=1)

In [42]:
X_test_all_feat.shape

(125207, 8)

In [43]:
X_test_all_feat.head(2)

,Id,year,team1,team2,mean_home_score,mean_away_score,percentage_wins_home,percentage_wins_away
0,0,3021,363,161,250.0,278.0,0.428571,0.535826
1,1,3021,286,2,271.0,242.0,0.593567,0.394464


In [44]:
X_test_not_scaled = X_test_all_feat.drop(['Id', 'team1', 'team2', 'year', 'mean_home_score', 'mean_away_score'], axis=1)

In [45]:
X_test_not_scaled.head(2)

,percentage_wins_home,percentage_wins_away
0,0.428571,0.535826
1,0.593567,0.394464


## X_train_not_scaled – преобразованная train.csv, но не нормированный year
## y_train – target from train.csv
## X_test_not_scaled – test.csv, но не нормированный year

# Нормировка 

In [46]:
scaler = StandardScaler()
train_scaled = pd.DataFrame(scaler.fit_transform(X_train_not_scaled), columns=list(X_train_not_scaled.columns))
test_scaled = pd.DataFrame(scaler.transform(X_test_not_scaled), columns=list(X_test_not_scaled.columns))


In [47]:
train_scaled.head(2)

,percentage_wins_home,percentage_wins_away
0,1.573942,1.261859
1,0.019886,0.518128


In [48]:
test_scaled.head(2)

,percentage_wins_home,percentage_wins_away
0,-0.609021,0.329087
1,0.778944,-0.935227


In [49]:
X_train = pd.concat([train_scaled, train_teams], axis=1)

In [50]:
print 'X_train.shape = ', X_train.shape

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-50-5c471cee1023>, line 1)

In [ ]:
X_train.head(3)

In [51]:
X_test = pd.concat([test_scaled, test_teams], axis=1)

In [52]:
print 'X_test.shape = ', X_test.shape

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-52-d0c02c7842cc>, line 1)

In [53]:
X_test.head(3)

,percentage_wins_home,percentage_wins_away,0,1,2,3,4,5,6,7,...,343,344,345,346,347,348,349,350,351,352
0,-0.609021,0.329087,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.778944,-0.935227,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.415097,-2.705856,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# cross_validation

In [54]:
train_data, test_data, train_labels, test_labels = cross_validation.train_test_split(X_train, y_train,
                                                                                     test_size=0.2, stratify = y_train)

In [57]:
print np.sum(test_labels['target'] == True) / np.sum(test_labels['target'] == False)

SyntaxError: invalid syntax (<ipython-input-57-d26796186182>, line 1)

In [58]:
print np.sum(train_labels['target'] == True) / np.sum(train_labels['target'] == False)

SyntaxError: invalid syntax (<ipython-input-58-01f60a776e31>, line 1)

In [59]:
print 'train_data.shape = ', train_data.shape
print 'test_data.shape = ', test_data.shape
print 'train_labels.shape = ', train_labels.shape
print 'test_labels.shape = ', test_labels.shape

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-59-955fe5902030>, line 1)

# XGB

## Первое приближение

In [237]:
train_data = X_train_hot_encod

In [238]:
train_labels = y_train

In [191]:
params = {  'max_depth': np.linspace(2, 12, 3, dtype=int), 
            'max_delta_step': np.linspace(0.03, 0.5, 3),
            'n_estimators': np.linspace(80,160, 3, dtype=int)}

In [192]:
xgb_classifier = xgboost.XGBClassifier()

In [195]:
y_t = np.array(train_labels).reshape(train_data.shape[0])
y_t.shape

(101609,)

# Linear_Classifier

In [251]:
lin_cl = linear_model.LogisticRegression(C=0.9, max_iter=130, tol=0.00005)

In [252]:
X_train.values

array([[ 1.57394169,  1.26185927,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.01988568,  0.51812785,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.21716998, -0.39299809,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 1.80204883,  0.39996197,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.15801178,  1.3317194 ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.13204259,  0.5177795 ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [253]:
y_train.values.T[0]

array([ True,  True,  True, ..., False,  True, False], dtype=bool)

In [254]:
lin_cl.fit(X_train.values, y_train.values.T[0])

LogisticRegression(C=0.9, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=130, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=5e-05,
          verbose=0, warm_start=False)

In [100]:
lin_cl.coef_

array([[  9.72915782e-01,  -9.54977420e-01,   7.96869136e-01,
         -7.86489228e-02,   3.66670561e-01,  -1.07892943e+00,
         -1.16318854e+00,  -3.65171675e-01,  -6.90008902e-01,
         -3.73642975e-01,  -3.03503957e-01,   2.17600337e-01,
          7.51766716e-01,  -3.30096795e-01,  -4.31851656e-01,
          4.49955796e-01,  -3.21480475e-01,   1.20327091e-01,
          7.52335429e-01,  -4.24211723e-01,  -1.38261440e-01,
          8.64427400e-01,  -1.19872639e+00,  -9.03058713e-01,
         -2.85211266e-01,  -1.38878254e+00,  -1.24990384e-01,
          6.55792857e-01,  -7.79832010e-01,  -5.23592542e-02,
          3.41407197e-01,   2.56225650e-01,  -3.53951788e-01,
         -1.32952168e-01,  -7.93839571e-01,   1.73674808e-01,
         -5.68258967e-01,   4.13160692e-02,   5.84697103e-01,
          3.07544727e-01,   5.65284874e-01,  -2.43245894e-01,
          4.11157053e-02,   1.22290315e-01,   1.79450933e-01,
         -7.14292322e-01,  -1.53840390e-01,   3.03761625e-01,
        

In [238]:
log_loss(test_labels, lin_cl.predict_proba(test_data)[:,1])

0.5982373982063619

In [104]:
params = { 'tol': [0.00001, 0.00005, 0.0001, 0.0005],
            'C': [0.3, 0.5, 0.7, 0.9, 1.0, 1.1, 1.3, 1.5],
            'max_iter': [50, 100, 150, 200, 75]
}

In [113]:
opt = GridSearchCV(estimator=lin_cl, param_grid=params, scoring='log_loss', cv=5)

In [114]:
opt.fit(X_train.values, y_train.values.T[0])

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.3, 0.5, 0.7, 0.9, 1.0, 1.1, 1.3, 1.5], 'tol': [1e-05, 5e-05, 0.0001, 0.0005], 'max_iter': [50, 100, 150, 200, 75]},
       pre_dispatch='2*n_jobs', refit=True, scoring='log_loss', verbose=0)

In [115]:
opt.best_score_

-0.62227389549332413

In [116]:
opt.best_params_

{'C': 0.3, 'max_iter': 50, 'tol': 0.0005}

In [117]:
y_pred = opt.predict_proba(X_test)[:,1]

In [145]:
len(y_pred)

20322

In [175]:
lin_cl = linear_model.LogisticRegression(C=0.8, tol=0.0005)

In [176]:
lin_cl.fit(train_data, train_labels)

//anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=0.8, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0005,
          verbose=0, warm_start=False)

In [255]:
y_pred = lin_cl.predict_proba(X_test)[:,1]

In [248]:
c

ValueError: Found arrays with inconsistent numbers of samples: [ 30483 125207]

In [169]:
k = 0
for i in range(len(y_pred)):
    if y_pred[i] > 0.975:
        k+=1
        y_pred[i] = 1
    if y_pred[i] < 0.025:
        k+=1
        y_pred[i]  = 0
print(k)

96


In [249]:
log_loss(test_labels, y_pred)

ValueError: Found arrays with inconsistent numbers of samples: [ 30483 125207]

## XGBOOST

In [61]:
train_data, test_data, train_labels, test_labels = cross_validation.train_test_split(X_train, y_train,
                                                                                     test_size=0.2
                                                                                    )

In [180]:
import xgboost

In [223]:
param = {}
#param['max_depth'] = 6
param['alpha'] = 10
param['booster'] = 'gblinear'
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'logloss'
param['eta'] = 0.001

numround = 1000

In [224]:
train_data.columns = [i for i in range(train_data.shape[1])]
test_data.columns = [i for i in range(test_data.shape[1])]

In [225]:
Xdatatrain = xgboost.DMatrix(data = train_data, label = train_labels)
Xdatatest = xgboost.DMatrix(data = test_data, label = test_labels)

plst = list(param.items())
watchlist = [(Xdatatrain, 'train'), (Xdatatest, 'eval')]            

bst = xgboost.train(plst, Xdatatrain, numround, evals = watchlist, verbose_eval = 10)
# ypredxgb_tr = bst.predict(Xdatatrain)

[0]	train-logloss:0.692749	eval-logloss:0.693028
[10]	train-logloss:0.691166	eval-logloss:0.691320
[20]	train-logloss:0.689417	eval-logloss:0.689707
[30]	train-logloss:0.687718	eval-logloss:0.688146
[40]	train-logloss:0.686061	eval-logloss:0.686628
[50]	train-logloss:0.684466	eval-logloss:0.685151
[60]	train-logloss:0.682898	eval-logloss:0.683717
[70]	train-logloss:0.681385	eval-logloss:0.682329
[80]	train-logloss:0.679901	eval-logloss:0.680970
[90]	train-logloss:0.678462	eval-logloss:0.679654
[100]	train-logloss:0.677077	eval-logloss:0.678376
[110]	train-logloss:0.675702	eval-logloss:0.677134
[120]	train-logloss:0.674385	eval-logloss:0.675918
[130]	train-logloss:0.673089	eval-logloss:0.674750
[140]	train-logloss:0.671829	eval-logloss:0.673606
[150]	train-logloss:0.670611	eval-logloss:0.672488
[160]	train-logloss:0.669422	eval-logloss:0.671405
[170]	train-logloss:0.668257	eval-logloss:0.670350
[180]	train-logloss:0.667129	eval-logloss:0.669327
[190]	train-logloss:0.666034	eval-logloss:

125207

In [265]:
k = 0
for i in range(len(y_pred)):
    if y_pred[i] > 0.97:
        y_pred[i] = 1
        k += 1
    if y_pred[i] < 0.03:
        y_pred[i] = 1
        k+= 1
print(k)

3142


In [62]:
from sklearn.svm import SVC

In [63]:
clf_svm = SVC(C=0.8)

In [64]:
train_data, test_data, train_labels, test_labels

(        percentage_wins_home  percentage_wins_away    0    1    2    3    4  \
 1233               -0.078023             -0.264007  0.0  0.0  0.0  0.0  0.0   
 55959               0.111260             -0.862480  0.0  0.0  0.0  0.0  0.0   
 39558               2.217145              0.112668  0.0  0.0  0.0  0.0  0.0   
 95153              -2.751238             -1.247483  0.0  0.0  0.0  0.0  0.0   
 50277              -2.303576             -0.332934  0.0  0.0  0.0  0.0  0.0   
 101026             -2.016454             -0.759233  0.0  0.0  0.0  0.0  0.0   
 100125             -0.775477              1.058346  0.0  0.0  0.0  0.0  0.0   
 95110              -0.008155             -0.992503  0.0  0.0  0.0  0.0  0.0   
 56177              -0.008155             -0.688783  0.0  0.0  0.0  0.0  0.0   
 47826              -0.103438             -0.005709  0.0  0.0  0.0  0.0  0.0   
 39959               0.562384              1.369685  0.0  0.0  0.0  0.0  0.0   
 91722               0.019517           

array([False,  True, False, ..., False,  True,  True], dtype=bool)

In [72]:
clf_knn = KNeighborsClassifier(n_neighbors=35)

In [73]:
clf_knn.fit(train_data.values, train_labels.values.T[0])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=35, p=2,
           weights='uniform')

In [74]:
y_pred = clf_knn.predict_proba(test_data.values)[:, 1]

In [75]:
log_loss(test_labels, y_pred)

0.62503410211763488

In [76]:
y_pred = clf_knn.predict_proba(X_test)[:,1]

# Вывод ответа

In [77]:
ss = sample_submission.copy()
ss.target = y_pred
ss.to_csv('ans.csv', index=False)